In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/MA

/content/drive/MyDrive/MA


In [3]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [3]:
!pip install transformers==4.2.0
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

     |████████████████████████████████| 1.8MB 8.4MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
     |████████████████████████████████| 2.9MB 51.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b65e89d186750bb2e16c8069cadfd5834a33fe23b5ac15d447f18ae8a656c460
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
from transformers import BertForSequenceClassification, DistilBertTokenizerFast, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import time
import pickle
#import torch_xla
#import torch_xla.core.xla_model as xm

In [5]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#DEVICE = xm.xla_device()
print(DEVICE)

cuda


In [6]:
def read_train_and_test(path): #path: path of the whole dataset
  try:
    train = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
  except FileNotFoundError:
    data_csv = pd.read_csv('./labeled_data.csv', names=['label','tweet'],header=0)
    train=data_csv.sample(frac=0.8,random_state=200) #random state is a seed value
    test=data_csv.drop(train.index)
    train.to_csv('train.csv')
    test.to_csv('test.csv')
  return train, test

train_csv, test_csv = read_train_and_test('./labeled_data.csv')

In [7]:
def run_tokenizer(train_csv, test_csv, merge_label=False):
    tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-uncased') 

    def get_max_len(tokenizer, train_csv):
        tweets = train_csv.tweet.values
        max_length = 0
        for t in tweets:
          ids = tokenizer.encode(t)
          max_length = max(len(ids),max_length)
        return max_length

    max_length = get_max_len(tokenizer, train_csv)
    train_tweets, train_labels = train_csv.tweet.values, train_csv.label.values
    test_tweets, test_labels = test_csv.tweet.values, test_csv.label.values
    if merge_label == True:
       train_labels = [l if l ==0 else 1 for l in train_labels]
       test_labels = [l if l ==0 else 1 for l in test_labels]

    def tokenize_for_tweet(tokenizer, tweets, labels):
        input_ids = []
        attention_masks = []

        for t in tweets:
            input_dict = tokenizer.encode_plus(t, add_special_tokens=True, max_length=max_length, truncation=True, padding='max_length',return_tensors='pt')
            input_ids.append(input_dict['input_ids'])
            attention_masks.append(input_dict['attention_mask'])
        input_ids = torch.cat(input_ids,dim=0)
        attention_masks = torch.cat(attention_masks,dim=0)
        labels=torch.tensor(labels)
        dataset = TensorDataset(input_ids, attention_masks, labels)
        return dataset
        
    train_dataset = tokenize_for_tweet(tokenizer, train_tweets, train_labels)
    test_dataset = tokenize_for_tweet(tokenizer, test_tweets, test_labels)
    num_label = 3 if merge_label == False else 2
    return train_dataset, test_dataset, num_label

train_dataset, test_dataset, num_label = run_tokenizer(train_csv, test_csv, merge_label = True)

In [8]:
batch_size = 17

train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size)

test_dataloader = DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size)

In [9]:
bert_model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = num_label)
bert_model = bert_model.to(DEVICE)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.

In [10]:
def evaluate(model, best_acc):
    num_total, num_correct = 0, 0
    model.train(False)
    with torch.no_grad():
      eval_loss = 0

      for input, mask, label in test_dataloader:
          model.zero_grad()

          loss, output = model(input_ids=input.to(DEVICE), attention_mask=mask.to(DEVICE), labels=label.to(DEVICE), return_dict=False)

          predict_label = torch.argmax(output, dim=1)

          num_correct += (predict_label == label).sum().item()
          num_total += len(label)

          eval_loss += loss.item()

      acc = num_correct/num_total
      if acc > best_acc:
        best_acc = acc
        torch.save(model, 'model'+str(best_acc))

    print('eval_loss: {}, accuracy: {}'.format(eval_loss,acc))
    return best_acc


In [11]:
def training(model, epochs, optimizer, scheduler=None):
    best_acc = 0
    for e in range(epochs):
        print('training {} epoch...'.format(e+1))
        start_time = time.time()

        train_loss =0

        model.train(True)
        for input, mask, label in train_dataloader:
          
            model.zero_grad()

            loss, output = model(input_ids=input.to(DEVICE), attention_mask=mask.to(DEVICE), labels=label.to(DEVICE),return_dict=False)

            train_loss += loss.item()
            loss.backward()

            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
            optimizer.step()

            if scheduler:
              scheduler.step()

        sec = time.time()-start_time
        print('{} seconds used......'.format(sec))
        print("{} training finished! train loss: {}".format(e+1,train_loss))
        print('evaluating...')
        best_acc = evaluate(model, best_acc)   

In [12]:
optimizer = AdamW(bert_model.parameters(),lr = 2e-5, eps = 1e-8)

In [13]:
epochs = 4

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
training(bert_model, epochs, optimizer)

training 1 epoch...


In [ ]:
best_acc = 0
for e in range(epochs):
  print('training {} epoch...'.format(e+1))
  start_time = time.time()

  train_loss = 0

  bert_model.train(True)

  for batch in train_dataloader:
    input = batch[0].to(DEVICE)
    mask = batch[1].to(DEVICE)
    label = batch[2].to(DEVICE)

    bert_model.zero_grad()

    loss, output = bert_model(input_ids=input, attention_mask=mask, labels=label,return_dict=False)
    
    train_loss += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)
    
    optimizer.step()
    scheduler.step()

  num_total, num_correct = 0, 0
  bert_model.train(False)
  with torch.no_grad():
    eval_loss = 0

    for batch in test_dataloader:
      bert_model.zero_grad()

      input = batch[0].to(DEVICE)
      mask = batch[1].to(DEVICE)
      label = batch[2].to(DEVICE)

      loss, output = bert_model(input_ids=input, attention_mask=mask, labels=label,return_dict=False)

      predict_label = torch.argmax(output, dim=1)

      num_correct += (predict_label == label).sum().item()
      num_total += len(label)

      eval_loss += loss.item()

    acc = num_correct/num_total
    sec = time.time()-start_time
    if acc > best_acc:
      best_acc = acc
      torch.save(bert_model, 'bert_model.bert')
  
  print('train_loss: {}, eval_loss: {}, accuracy: {}'.format(train_loss,eval_loss,acc))
  print('{} seconds used......'.format(sec))
      
    


